# 2.1 - Web Scraping (bs4)

$$$$

![scraping](images/scraping.png)

$$$$

Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs. 

En los últimos años el web scraping se ha convertido en una técnica muy utilizada dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro último recurso a falta de una API o un feed RSS. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

### Extracción desde el HTML

Para scrapear una página web, en primer lugar debemos conocer las estructura que tiene el HTML. Veamos la estructura básica.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$

Las etiquetas pueden tener atributos. Por ejemplo:
 
`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>` 

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las etiquetas dentro del HTML de la página web.

Usaremos la herramienta **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [1]:
!pip install beautifulsoup4

In [2]:
import requests as req

from bs4 import BeautifulSoup as bs

### Ejemplos Wikipedia

**[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)**

In [3]:
url='https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

In [4]:
# usamos el requests para extraer el html string

html=req.get(url).content

html[:1000]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of European countries by life expectancy - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"20c643d3-91d3-4cb5-bf3d-9dcf245d2934","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_European_countries_by_life_expectancy","wgTitle":"List of European countries by life expectancy","wgCurRevisionId":1049636953,"wgRevisionId":1049636953,"wgArticleId":22175559,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short descripti

In [6]:
# bs4 para manejar el html

soup=bs(html, 'html.parser')

type(soup)

bs4.BeautifulSoup

In [8]:
# tabla

tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [17]:
# extraer las filas

filas=tabla.find_all('tr')

filas=[f.text.strip().split('\n') for f in filas]

filas[:5]

[['Rank',
  '',
  'Country',
  'Life expectancy[1]',
  '',
  'Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]'],
 ['1', '', '\xa0Monaco[3]', '', '89.4'],
 ['2', '', '\xa0San Marino[4]', '', '83.4'],
 ['3', '', '\xa0\xa0Switzerland', '83.0'],
 ['4', '', '\xa0Spain', '82.8', '', '56%']]

In [18]:
# minima limpieza

final=[]


for e in filas:
    tmp=[]

    for st in e:
        if st!='':
            tmp.append(st)
    final.append(tmp)
    
final[:5]

[['Rank',
  'Country',
  'Life expectancy[1]',
  'Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]'],
 ['1', '\xa0Monaco[3]', '89.4'],
 ['2', '\xa0San Marino[4]', '83.4'],
 ['3', '\xa0\xa0Switzerland', '83.0'],
 ['4', '\xa0Spain', '82.8', '56%']]

In [19]:
# dataframe

import pandas as pd

col_names=final[0]

data=final[1:]

df=pd.DataFrame(data, columns=col_names)

df.head(10)

,Rank,Country,Life expectancy[1],"Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]"
0,1,Monaco[3],89.4,None
1,2,San Marino[4],83.4,None
2,3,Switzerland,83.0,None
3,4,Spain,82.8,56%
4,5,Liechtenstein,82.7,28%
5,6,Italy,82.7,50%
6,7,Norway,82.5,38%
7,8,Iceland,82.5,47%
8,9,Luxembourg,82.3,38%
9,10,France,82.3,50%


### Ejemplo geolocalización por IP

https://tools.keycdn.com/geo

**¿Dónde estoy?**

In [20]:
url='https://tools.keycdn.com/geo'

In [21]:
html=req.get(url).content

soup=bs(html, 'html.parser')

In [24]:
soup.find('div', {'id': 'geoResult'})

<div class="mt-4" id="geoResult">
<div class="bg-light medium rounded p-3">
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mb-2">Location</p> <dl class="row mb-0">
<dt class="col-4">City</dt><dd class="col-8 text-monospace">Madrid</dd><dt class="col-4">Region</dt><dd class="col-8 text-monospace">Madrid (M)</dd><dt class="col-4">Postal code</dt><dd class="col-8 text-monospace">28050</dd><dt class="col-4">Country</dt><dd class="col-8 text-monospace">Spain (ES)</dd><dt class="col-4">Continent</dt><dd class="col-8 text-monospace">Europe (EU)</dd><dt class="col-4">Coordinates</dt><dd class="col-8 text-monospace">40.4163 (lat) / -3.6934 (long)</dd><dt class="col-4">Time</dt><dd class="col-8 text-monospace">2022-01-28 10:25:25 (Europe/Madrid)</dd> </dl>
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mt-4 mb-2">Network</p>
<dl class="row mb-0">
<dt class="col-4">IP address</dt><dd class="col-8 text-monospace">2.136.116

In [27]:
tabla=soup.find('div', {'id': 'geoResult'})

tabla.find_all('dd', {'class': 'col-8 text-monospace'})

[<dd class="col-8 text-monospace">Madrid</dd>,
 <dd class="col-8 text-monospace">Madrid (M)</dd>,
 <dd class="col-8 text-monospace">28050</dd>,
 <dd class="col-8 text-monospace">Spain (ES)</dd>,
 <dd class="col-8 text-monospace">Europe (EU)</dd>,
 <dd class="col-8 text-monospace">40.4163 (lat) / -3.6934 (long)</dd>,
 <dd class="col-8 text-monospace">2022-01-28 10:25:25 (Europe/Madrid)</dd>,
 <dd class="col-8 text-monospace">2.136.116.82</dd>,
 <dd class="col-8 text-monospace">82.red-2-136-116.staticip.rima-tde.net</dd>,
 <dd class="col-8 text-monospace">Telefonica De Espana</dd>,
 <dd class="col-8 text-monospace">3352</dd>]

In [28]:
conexion=[e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

conexion

['Madrid',
 'Madrid (M)',
 '28050',
 'Spain (ES)',
 'Europe (EU)',
 '40.4163 (lat) / -3.6934 (long)',
 '2022-01-28 10:25:25 (Europe/Madrid)',
 '2.136.116.82',
 '82.red-2-136-116.staticip.rima-tde.net',
 'Telefonica De Espana',
 '3352']

In [29]:
tabla.find_all('dt')

[<dt class="col-4">City</dt>,
 <dt class="col-4">Region</dt>,
 <dt class="col-4">Postal code</dt>,
 <dt class="col-4">Country</dt>,
 <dt class="col-4">Continent</dt>,
 <dt class="col-4">Coordinates</dt>,
 <dt class="col-4">Time</dt>,
 <dt class="col-4">IP address</dt>,
 <dt class="col-4">Hostname</dt>,
 <dt class="col-4">Provider</dt>,
 <dt class="col-4">ASN</dt>]

**Búsqueda según IP**

https://tools.keycdn.com/geo?host=137.255.90.7

In [30]:
url='https://tools.keycdn.com/geo?host=137.255.90.7'

html=req.get(url).content

soup=bs(html, 'html.parser')

In [31]:
tabla=soup.find('div', {'id': 'geoResult'})

tabla.find_all('dd', {'class': 'col-8 text-monospace'})

[<dd class="col-8 text-monospace">Benin (BJ)</dd>,
 <dd class="col-8 text-monospace">Africa (AF)</dd>,
 <dd class="col-8 text-monospace">9.5 (lat) / 2.25 (long)</dd>,
 <dd class="col-8 text-monospace">2022-01-28 10:31:17 (Africa/Porto-Novo)</dd>,
 <dd class="col-8 text-monospace">137.255.90.7</dd>,
 <dd class="col-8 text-monospace">137.255.90.7</dd>]

In [32]:
tabla.find_all('dt')

[<dt class="col-4">Country</dt>,
 <dt class="col-4">Continent</dt>,
 <dt class="col-4">Coordinates</dt>,
 <dt class="col-4">Time</dt>,
 <dt class="col-4">IP address</dt>,
 <dt class="col-4">Hostname</dt>]

In [34]:
'{:2.2f}'.format(12.5436363636363)  # formato en strings de numeros

'12.54'

In [33]:
list_ip=['137.255.90.7', '255.255.90.7', '177.255.21.7']

In [35]:
def geo(ip):
    
    url=f'https://tools.keycdn.com/geo?host={ip}'
    
    html=req.get(url).content

    soup=bs(html, 'html.parser')
    
    tabla=soup.find('div', {'id': 'geoResult'})
    
    conexion=[e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    return conexion

In [37]:
for ip in list_ip:
    print(geo(ip))

['Benin (BJ)', 'Africa (AF)', '9.5 (lat) / 2.25 (long)', '2022-01-28 10:35:53 (Africa/Porto-Novo)', '137.255.90.7', '137.255.90.7']
['255.255.90.7', '255.255.90.7']
['Cartagena', 'Departamento de Bolivar (BOL)', '130001', 'Colombia (CO)', 'South America (SA)', '10.4013 (lat) / -75.5267 (long)', '2022-01-28 04:35:53 (America/Bogota)', '177.255.21.7', 'Dinamic-Tigo-177-255-21-7.tigo.com.co', 'Colombia Movil', '27831']


### Ejemplo LinkedIn

In [83]:
import pandas as pd


def linkedin(num_pages, keywords, country, n_secs, exp):
    
    URL='https://www.linkedin.com/jobs/search/'

    data=[]

    for i in range(num_pages):

        scrape_url=''.join([
            URL,  # url base
            f'?keywords={keywords}',   # palabras clave de busqueda
            f'&location={country}',   # pais lugar
            f'&f_TPR={n_secs}',        # segundos atras
            f'&F_E={exp}',            # experiencia
            f'&start={i*25}'           # numero de paginas (i)
        ])


        html=req.get(scrape_url).content  # el html de la pagina

        soup=bs(html, 'html.parser')       # la sopa parseada

        for oferta in soup.find_all('div', 
                               class_="base-card base-card--link base-search-card base-search-card--link job-search-card"):
            # bucle para las ofertas
            titulo=oferta.find('span', class_="screen-reader-text").text.strip()   # titulo de la oferta

            nombre=oferta.find('a', class_="hidden-nested-link").text.strip()   # nombre de la compañia

            link_comp=oferta.find('a', class_="hidden-nested-link").attrs['href']  # link de la compañia

            lugar=oferta.find('span', class_="job-search-card__location").text.strip()  # lugar

            link_ofer=oferta.find('a', class_="base-card__full-link").attrs['href']   # link de la oferta

            fecha=oferta.find('time').attrs['datetime']       # fecha de publicacion

            data.append({
                'title': titulo,
                'name': nombre,
                'link de la compañia': link_comp,
                'location': lugar,
                'link de la oferta': link_ofer,
                'datetime': fecha
            })
            
    return pd.DataFrame(data)


In [84]:
linkedin(3, 'data', 'Madrid', 300000, 2)

,title,name,link de la compañia,location,link de la oferta,datetime
0,AUDITOR JUNIOR DATA ANALYTICS,Iberdrola,https://es.linkedin.com/company/iberdrola?trk=...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/auditor-juni...,2022-01-18
1,Financial Data Analyst & IT Systems,Mercedes-Benz Financial Services España,https://es.linkedin.com/company/mercedes-benz-...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/financial-da...,2022-01-25
2,Analistas de Datos,Auchan Retail,https://fr.linkedin.com/company/auchan?trk=pub...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/analistas-de...,2022-01-27
3,Business Data Analyst,"Headspring, by Financial Times & IE Business S...",https://uk.linkedin.com/company/goheadspring?t...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/business-dat...,2022-01-28
4,Consultor Data Science,Gain Dynamics,https://es.linkedin.com/company/gain-dynamics?...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/consultor-da...,2022-01-26
...,...,...,...,...,...,...
69,Creative Data Analyst,Fever,https://www.linkedin.com/company/fever-up?trk=...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/creative-dat...,2022-01-24
70,Chief Data Officer,Spring Professional,https://ch.linkedin.com/company/spring-profess...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/chief-data-o...,2022-01-19
71,Data Scientist-Cliente final,IDavinci,https://es.linkedin.com/company/idavinci?trk=p...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/data-scienti...,2022-01-24
72,Data scientist Junior,METIORA,https://es.linkedin.com/company/metiora?trk=pu...,"Madrid, Community of Madrid, Spain",https://es.linkedin.com/jobs/view/data-scienti...,2021-12-29
